In [1]:
import data_engine as de
import pandas as pd
from feature_engine.discretisation import EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer,CategoricalImputer

In [2]:
data_save_path = './data/'
root_path = r'C:\\Users\\8prab\\Documents\\NSE_Equity_Futures_iEOD\\'
pattern = '*BANKNIFTY.txt'
data_name = 'banknifty'
#base_df= de.create_dataset(root_path,pattern,data_save_path,data_name)
base_df = pd.read_csv(r'C:\Users\8prab\Google Drive\Work\trading\data\banknifty\banknifty.csv')

In [3]:
base_df.shape

(1170729, 4)

In [4]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('labelgenerator_1', de.LabelCreator(freq='1min',shift=-15,shift_column='close')),('labelgenerator_2', de.LabelCreator(freq='1min',shift=-30,shift_column='close')),('mean_imputer', MeanMedianImputer()),('cat_imputer', CategoricalImputer())])
base_df = pipe.fit(base_df).transform(base_df)

In [5]:
base_df.head()

,open,high,low,close,label_-15_1min_close,label_-30_1min_close
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,40to60,40to60
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,40to60,20to40
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,40to60,40to60
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,40to60,20to40
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,60to80,40to60


In [9]:
obj = EqualWidthDiscretiser(variables=['open'], bins=10, return_object=False, return_boundaries=False)
obj.fit(base_df.fillna(0)).transform(base_df)

,open,high,low,close,label_-15_1min_close,label_-30_1min_close
2010-03-02 09:01:00,1,8856.95,8826.90,8839.80,40to60,40to60
2010-03-02 09:02:00,1,8853.90,8832.80,8852.30,40to60,20to40
2010-03-02 09:03:00,1,8855.70,8844.65,8846.65,40to60,40to60
2010-03-02 09:04:00,1,8850.70,8843.75,8846.70,40to60,20to40
2010-03-02 09:05:00,1,8846.40,8837.75,8841.60,60to80,40to60
...,...,...,...,...,...,...
2022-01-13 15:28:00,9,38469.15,38459.45,38467.20,unknown,unknown
2022-01-13 15:29:00,9,38479.85,38456.65,38456.65,unknown,unknown
2022-01-13 15:30:00,9,38471.55,38455.15,38465.70,unknown,unknown
2022-01-13 15:31:00,9,38466.05,38466.05,38466.05,unknown,unknown


In [8]:
pipe.get_params()

{'memory': None,
 'steps': [('labelgenerator_1', LabelCreator()),
  ('labelgenerator_2', LabelCreator(shift=-30)),
  ('technical', TechnicalIndicator())],
 'verbose': False,
 'labelgenerator_1': LabelCreator(),
 'labelgenerator_2': LabelCreator(shift=-30),
 'technical': TechnicalIndicator(),
 'labelgenerator_1__freq': '1min',
 'labelgenerator_1__shift': -15,
 'labelgenerator_1__shift_column': 'close',
 'labelgenerator_2__freq': '1min',
 'labelgenerator_2__shift': -30,
 'labelgenerator_2__shift_column': 'close'}

In [9]:
pipe.set_params()

Pipeline(steps=[('labelgenerator_1', LabelCreator()),
                ('labelgenerator_2', LabelCreator(shift=-30)),
                ('technical', TechnicalIndicator())])

In [10]:
from feature_engine.discretisation import EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer

In [11]:
base_df.columns

Index(['open', 'high', 'low', 'close', 'label_-15_1min_close',
       'label_-30_1min_close', 'momentum_ADX', 'momentum_ADXR', 'momentum_APO',
       'momentum_BOP',
       ...
       'overlap_double_exp_moving_avg', 'overlap_exp_moving_avg',
       'overlap_hilbert_transform', 'overlap_kaufman_adaptive_moving_avg',
       'overlap_moving_avg', 'overlap_midpoint', 'overlap_midpoint_price',
       'overlap_simple_moving_avg', 'overlap_triangular_moving_average',
       'overlap_weighted_moving_average'],
      dtype='object', length=141)

ValueError: Some of the variables to transform contain inf values. Check and remove those before using this transformer.

In [29]:
n = 2
x = np.concatenate([[np.nan] * (n), base_df['close'].values])

def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
arr = rolling_window(x, n + 1)

base_df['new'] = (arr[:, :-1] > arr[:, [-1]]).sum(axis=1)

<ipython-input-29-402c5888b12e>:10: RuntimeWarning: invalid value encountered in greater
  base_df['new'] = (arr[:, :-1] > arr[:, [-1]]).sum(axis=1)


In [30]:
base_df

,open,high,low,close,label_-15_1min_close,label_-30_1min_close,new
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,40to60,40to60,0
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,40to60,20to40,0
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,40to60,40to60,1
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,40to60,20to40,1
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,60to80,40to60,2
...,...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,unknown,unknown,1
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,unknown,unknown,2
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,unknown,unknown,1
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,unknown,unknown,0


In [25]:
import numpy as np
df = pd.DataFrame({"a": np.random.randint(1, high=10, size=50)})
#print(df)
def f(s):
    #d = s.to_frame().groupby(pd.cut(s, np.arange(s.min(), s.max()*1.01, 1), include_lowest=True)).transform("count")
    tmp, bins = pd.cut(
                x=s, bins=3, retbins=True, duplicates="drop"
            )
    bins = list(bins)
    bins[0] = float("-inf")
    bins[len(bins) - 1] = float("inf")
    return pd.cut(x=s, bins=bins, labels=False)

df["a"].rolling(10).apply(f)

TypeError: cannot convert the series to <class 'float'>

In [21]:
tmp, bins = pd.cut(
                x=df['a'], bins=3, retbins=True, duplicates="drop"
            )
bins

array([0.992     , 3.66666667, 6.33333333, 9.        ])

In [22]:
bins = list(bins)
bins[0] = float("-inf")
bins[len(bins) - 1] = float("inf")
bins

[-inf, 3.6666666666666665, 6.333333333333333, inf]

In [24]:
t = pd.cut(x=df['a'], bins=bins, labels=False)
t

0     0
1     2
2     2
3     2
4     0
5     1
6     2
7     2
8     1
9     2
10    0
11    0
12    2
13    2
14    0
15    1
16    1
17    1
18    1
19    0
20    1
21    0
22    0
23    2
24    1
25    1
26    2
27    2
28    1
29    1
30    2
31    0
32    0
33    2
34    2
35    1
36    0
37    1
38    2
39    0
40    1
41    1
42    0
43    0
44    1
45    0
46    1
47    2
48    1
49    0
Name: a, dtype: int64